In [ ]:
import pandas as pd

df = pd.read_csv('netflix_titles.csv')
#df = df.drop(columns=['UserName', 'ScreenName', 'Location', 'TweetAt'])
df = df.drop(columns=['show_id','country', 'title','director', 'cast', 'date_added', 'release_year','duration', 'rating'])
df.head()

,type,listed_in,description
0,TV Show,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,Movie,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,Movie,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,Movie,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,Movie,Dramas,A brilliant group of students become card-coun...


In [ ]:
import nltk, os, re, string

from keras.layers import Input, LSTM, Bidirectional, SpatialDropout1D, Dropout, Flatten, Dense, Embedding, BatchNormalization
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
#from keras.utils import to_binary

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# lower-case all characters
df.type = df.type.apply(lambda x: x.lower())
df.listed_in = df.listed_in.apply(lambda x: x.lower())
df.description = df.description.apply(lambda x: x.lower())

# removing functuation
def cleaner(data):
    return(data.translate(str.maketrans('','', string.punctuation)))
    df.type = df.type.apply(lambda x: cleaner(x))
    df.listed_in = df.listed_in.apply(lambda x: lem(x))
    df.description = df.description.apply(lambda x: lem(x))

## lematization
lemmatizer = WordNetLemmatizer()

def lem(data):
    pos_dict = {'N': wn.NOUN, 'V': wn.VERB, 'J': wn.ADJ, 'R': wn.ADV}
    return(' '.join([lemmatizer.lemmatize(w,pos_dict.get(t, wn.NOUN)) for w,t in nltk.pos_tag(data.split())]))
    df.type = df.type.apply(lambda x: lem(x))
    df.listed_in = df.listed_in.apply(lambda x: lem(x))
    df.description = df.description.apply(lambda x: lem(x))


# removing number
def rem_numbers(data):
    return re.sub('[0-9]+','',data)
    df['type'].apply(rem_numbers)
    df['listed_in'].apply(rem_numbers)
    df['description'].apply(rem_numbers)

# removing stopword
st_words = stopwords.words()
def stopword(data):
    return(' '.join([w for w in data.split() if w not in st_words ]))
    df.type = df.OriginalTweet.apply(lambda x: stopword(x))
    df.listed_in = df.listed_in.apply(lambda x: lem(x))
    df.description = df.description.apply(lambda x: lem(x))

In [ ]:
df.head()

,type,listed_in,description
0,tv show,"international tv shows, tv dramas, tv sci-fi &...",in a future where the elite inhabit an island ...
1,movie,"dramas, international movies",after a devastating earthquake hits mexico cit...
2,movie,"horror movies, international movies","when an army recruit is found dead, his fellow..."
3,movie,"action & adventure, independent movies, sci-fi...","in a postapocalyptic world, rag-doll robots hi..."
4,movie,dramas,a brilliant group of students become card-coun...


In [ ]:
category = pd.get_dummies(df.type)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='type')
df_baru

,listed_in,description,movie,tv show
0,"international tv shows, tv dramas, tv sci-fi &...",in a future where the elite inhabit an island ...,0,1
1,"dramas, international movies",after a devastating earthquake hits mexico cit...,1,0
2,"horror movies, international movies","when an army recruit is found dead, his fellow...",1,0
3,"action & adventure, independent movies, sci-fi...","in a postapocalyptic world, rag-doll robots hi...",1,0
4,dramas,a brilliant group of students become card-coun...,1,0
...,...,...,...,...
7782,"dramas, international movies",when lebanon's civil war deprives zozo of his ...,1,0
7783,"dramas, international movies, music & musicals",a scrappy but poor boy worms his way into a ty...,1,0
7784,"documentaries, international movies, music & m...","in this documentary, south african rapper nast...",1,0
7785,"international tv shows, reality tv",dessert wizard adriano zumbo looks for the nex...,0,1


In [ ]:
description = df_baru['description'].values + '' + df_baru['listed_in'].values
label = df_baru[['movie', 'tv show']].values

In [ ]:
from sklearn.model_selection import train_test_split
description_latih, description_test, label_latih, label_test = train_test_split(description, label, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(description_latih)
tokenizer.fit_on_texts(description_test)

sekuens_latih = tokenizer.texts_to_sequences(description_latih)
sekuens_test = tokenizer.texts_to_sequences(description_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [ ]:
from tensorflow.python.keras.backend import dropout
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#class myCallback(tf.keras.callbacks.Callback):
  #def on_epoch_end(self, epoch, logs={}):
    #if(logs.get('accuracy')>0.9 and logs.get('val_accuracy')) :
      #print("\nAkurasi dan Val Akurasi telah mencapai >90%!")
      #self.model.stop_training = True
#callbacks = myCallback()

In [ ]:
#history = model.fit(padded_latih, label_latih, epochs=50,
                    #validation_data=(padded_test, label_test), verbose=2) #callbacks=[callbacks], validation_steps=30)
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs,
                    validation_data=(padded_test, label_test), verbose=2)
#num_epochs = 30
#history = model.fit(padded_latih, label_latih, epochs=num_epochs,
                    #validation_data=(padded_test, label_test), verbose=2]#, callbacks=[callbacks])

Epoch 1/30
195/195 - 23s - loss: 0.1758 - accuracy: 0.9209 - val_loss: 0.0085 - val_accuracy: 0.9968
Epoch 2/30
195/195 - 10s - loss: 0.0020 - accuracy: 0.9998 - val_loss: 0.0025 - val_accuracy: 0.9994
Epoch 3/30
195/195 - 11s - loss: 0.0017 - accuracy: 0.9998 - val_loss: 0.0026 - val_accuracy: 0.9987
Epoch 4/30
195/195 - 10s - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0034 - val_accuracy: 0.9987
Epoch 5/30
195/195 - 10s - loss: 1.1469e-04 - accuracy: 1.0000 - val_loss: 0.0035 - val_accuracy: 0.9987
Epoch 6/30
195/195 - 10s - loss: 3.5929e-05 - accuracy: 1.0000 - val_loss: 0.0044 - val_accuracy: 0.9987
Epoch 7/30
195/195 - 10s - loss: 2.6006e-05 - accuracy: 1.0000 - val_loss: 0.0059 - val_accuracy: 0.9987
Epoch 8/30
195/195 - 11s - loss: 1.2254e-05 - accuracy: 1.0000 - val_loss: 0.0058 - val_accuracy: 0.9987
Epoch 9/30
195/195 - 12s - loss: 8.0544e-06 - accuracy: 1.0000 - val_loss: 0.0067 - val_accuracy: 0.9987
Epoch 10/30
195/195 - 11s - loss: 8.1094e-06 - accuracy: 1.0000 - val_l